


<font size='10' color = 'E3A440'>**Mégadonnées et techniques avancées démystifiées**</font>
=======
<font color = 'E3A440'>*Nouvelles méthodes d’analyse et leur implication quant à la gestion des mégadonnées en SSH (partie 1)*</font>
=============


Cet atelier s’inscrit dans le cadre de la formation [Mégadonnées et techniques avancées démystifiées](https://www.4point0.ca/2022/08/22/formation-megadonnees-demystifiees/) (séance 6).

Les sciences humaines et sociales sont souvent confrontées à l’analyse de données non structurées, comme le texte. Après avoir préparé les données, plusieurs techniques d’analyse venant de l’apprentissage automatique peuvent être utilisées. Pendant cet atelier, les participants seront initiés aux méthodes supervisées et non supervisées à des buts d’analyse avec Python.

Note : Cet atelier se poursuit lors d’une 2e séance le 10 novembre.

Structure de l'atelier :
1. Presentation of sections 1 and 2 in a plenary mode (20 minutes)
2. Individual work on section 3 (20 minutes)
3. Group work on section 4 (60 minutes)
4. Plenary session with groups presentations (20 minutes)

Ce tutoriel ne peut pas être consideré exaustif .... 

### Authors: 
- Bruno Agard <bruno.agard@polymtl.ca>
- Davide Pulizzotto <davide.pulizzotto@polymtl.ca>

### Table of Contents
Bruno Agard

Département de Mathématiques et de génie Industriel

École Polytechnique de Montréal

# <font color = 'E3A440'>0. Préparation environnement </font>

In [1]:
# Downloading of data from the GitHub project
!rm -rf Donnees_demystifiees_seance_6/
!git clone https://github.com/puli83/Donnees_demystifiees_seance_6

Cloning into 'Donnees_demystifiees_seance_6'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 75 (delta 26), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [2]:
# Import modules
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import os
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

# <font color = 'E3A440'>1. *Préparation des données textulles*</font>

L'analyse de données textuelles implique la transformation d'un texte en un objet mathematique qui peut être utilisé par des algorithmes et des modèles statistique. Cette étape est importante car permet de **structurer** des données non structurées, comme le texte.


###  <font color = 'E3A440'>**1.1 Étapes fondamentales du prétraitement**</font>

Prenons une phrase pour decortiquer les passages que nous pemrettent de la transformer en information structurée.

In [189]:
sentence = """At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good."""

In [190]:
len(sentence)

78

#### <font color = 'E3A440'>*a. Tokenisation*</font>

Cette étape consiste à couper la phrase en unités linguistiques élémantaire et dotées de sens, ce qui est gnralement appelé le "mot".

Dans le module `nltk`, il existe une fonction qui permet cette opération, soit `word_tokenize()`.

In [191]:
# La function word_tokenize() prend la phrase comme argument.
words = nltk.word_tokenize(sentence)
print(words)
len(words)

['At', 'eight', "o'clock", ',', 'on', 'Thursday', 'morning', ',', 'the', 'great', 'Arthur', 'did', "n't", 'feel', 'VERY', 'good', '.']


17

#### <font color = 'E3A440'>*b. Analyse morphosyntaxique*</font>

Après avoir identifé tous les mots, il est possible de analyser leur rôle morphosyntaxique, à des fins d'analyse et/ou filtrage. 

In [192]:
 # La function word_tokenize() prend la liste de mots comme argument.
words_pos = nltk.pos_tag(words, tagset='universal')
print(words_pos)
len(words_pos)

[('At', 'ADP'), ('eight', 'NUM'), ("o'clock", 'NOUN'), (',', '.'), ('on', 'ADP'), ('Thursday', 'NOUN'), ('morning', 'NOUN'), (',', '.'), ('the', 'DET'), ('great', 'ADJ'), ('Arthur', 'NOUN'), ('did', 'VERB'), ("n't", 'ADV'), ('feel', 'VERB'), ('VERY', 'ADV'), ('good', 'ADJ'), ('.', '.')]


17

Voici la liste de possible POS tags:

| **POS** | **DESCRIPTION**           | **EXAMPLES**                                      |
| ------- | ------------------------- | ------------------------------------------------- |
| ADJ     | adjective                 | big, old, green, incomprehensible, first      |
| ADP     | adposition                | in, to, during                                |
| ADV     | adverb                    | very, tomorrow, down, where, there            |
| AUX     | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ    | conjunction               | and, or, but                                  |
| CCONJ   | coordinating conjunction  | and, or, but                                  |
| DET     | determiner                | a, an, the                                    |
| INTJ    | interjection              | psst, ouch, bravo, hello                      |
| NOUN    | noun                      | girl, cat, tree, air, beauty                  |
| NUM     | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART    | particle                  | ’s, not                                      |
| PRON    | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN   | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT   | punctuation               | ., (, ), ?                                    |
| SCONJ   | subordinating conjunction | if, while, that                               |
| SYM     | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :)               |
| VERB    | verb                      | run, runs, running, eat, ate, eating          |
| X       | other                     | sfpksdpsxmsa                                  |
| SPACE   | space                     |                                                   |


#### <font color = 'E3A440'>*c. Retirer la ponctuation*</font>

Une autre opération consiste à retirer la ponctuation. Ce type de filtrage reduit le nombre de sugne graphique qui participent le moin à la constructuion de la sémantique de la prhase. 
Dans certain contexte, comme en stylometrie, ce processus est appliquée avec de terchniques plus sofistiquées. 

In [ ]:
# La ligne de code suivant itére sur chaque signe graphique et retient ceux qui contiennet de caracteres alphanumérique.
words_pos = [(w, pos) for w, pos in words_pos if w.isalnum()]
print(words_pos)
len(words_pos)
# Ikl est possible aussi d'utiilser le résultat de l'analyse morphosyntaxique pour éliminer la ponctuaction
# words_pos = [(w, pos) for w, pos in words_pos if pos != '.']
# print(words_pos)

[('At', 'ADP'), ('eight', 'NUM'), ('on', 'ADP'), ('Thursday', 'NOUN'), ('morning', 'NOUN'), ('the', 'DET'), ('great', 'ADJ'), ('Arthur', 'NOUN'), ('did', 'VERB'), ('feel', 'VERB'), ('VERY', 'ADV'), ('good', 'ADJ')]


12

#### <font color = 'E3A440'>*d. Convertir chaque caractère en minuscule*</font>

Cette étape constitue une première opéraiton de normalisation des mots et leur réduction à une forme graphique unique. Ce genre d'étape permet de regrouper chaque occurence d'un mot sous une seule forme.

In [ ]:
# La ligne de code suivant itére sur chaque signe graphique et le transforme en minuscule.
words_pos = [(w.lower(), pos) for w, pos in words_pos]
print(words_pos)

[('at', 'ADP'), ('eight', 'NUM'), ('on', 'ADP'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('the', 'DET'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('did', 'VERB'), ('feel', 'VERB'), ('very', 'ADV'), ('good', 'ADJ')]


#### <font color = 'E3A440'>*e. Retirer les stopwords (mots vides)*</font>

Une autre opération de filtrage constitue dan l'élimination de mots fonctionnels. Cette liste de mots contient tout les connecteurs de phrase, comme "et", "mais", "toutefois" et de mots avec faible valeur sémantique, comem les verbes modaux. 
Comme d'autres opéraiton de filtrage, l'enjeuy est celui de nettoyer le plus possible le vocabulaire et de reduyire toutes les occurrences d'un mot sous une forme graphique unique.

In [ ]:
# Nous importons la liste de stopword en anglais
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# La ligne de code suivant itére sur chaque signe graphique et garde ce qui n sont pas dans la liste de stopword.
words_pos = [(w, pos) for w, pos in words_pos if w not in stopwords.words("english")]
print(words_pos)
len(words_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


7

#### <font color = 'E3A440'>*f. Rammener les mots à leur racine*</font> 

En suivant le même objectif, nous retirons le suffixe morphologique des mots, ce qui augmente le niveau de réduction de chaque occurrence d'un mot à une unique forme graphique.

Ils existent deux méthode fondamentales: la racinisaiton et la lemmatisation.
La première reduit les occurence à une racine qui est inférée au moyen de plusieur techniques, l'autre est la réduciton de l'occurrence à son lemme. 

In [ ]:
# Racinisation: technique Porter
from nltk.stem.porter import PorterStemmer
stemmed_pos = [(PorterStemmer().stem(w), pos) for w, pos in words_pos]
print(stemmed_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morn', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


In [ ]:
# Racinisation: technique Lancaster
from nltk.stem import LancasterStemmer
stemmed_pos = [(LancasterStemmer().stem(w), pos) for w, pos in words_pos]
print(stemmed_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morn', 'NOUN'), ('gre', 'ADJ'), ('arth', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


In [ ]:
# Lemmatisaiton: utilisant le thesaurus wordnet
from nltk.stem.wordnet import WordNetLemmatizer
lemmed_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
print(lemmed_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


#### <font color = 'E3A440'>*g. Filtrage selon le rôle morphosyntaxique*</font>

Le filtrage des unités lexicales peut s'étendre jusqu'à l'élimination d'unités qui ne font pas partie d'une liste de rôles morphosyntaxique prédéfinie. 

In [ ]:
# Retenir seulement les noms et les adjectifs
lemmed_pos = [(w, pos) for w, pos in words_pos if pos in ['NOUN','ADJ']]
print(lemmed_pos)

[('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('good', 'ADJ')]


## <font color = 'E3A440'>**1.2 Traitement d'un corpus**</font>

Le prétraitement d'un corpus de recherche peut mettre en place plusieurs autres étapes. La plus importante est la **segmentation**. 

### <font color = 'E3A440'>*1. Découpage du texte*</font>

Tout dépendant de l'objectif de l'analyse, le texte peut être découpé en plusieurs fragments, chacun desquels peut être un document, un paragraphe, une concordance, un groupe de phrases, une phrase simple, un syntagme, etc.



In [261]:
text = """At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good.
          The following morning, at nine, Arthur felt better.
          A dog run in the street."""
len(text)

175

Dans le bloc de code suivant, nous faisons une segmentation par pharse.

In [265]:
sentences = nltk.sent_tokenize(text)
print(sentences)
len(sentences)

["At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good.", 'The following morning, at nine, Arthur felt better.', 'A dog run in the street.']


3

### <font color = 'E3A440'>*2. Annotation et nettoyage*</font>

Les opérations précedentes d'annotation morphosyntaxique et de filtrage seront appliquées à chaque fragement du corpus qui a été créé.


#### <font color = 'E3A440'>*a. Création d'une fonction*</font>

Dans le code suivant, une fonction est créé pour englober toutes les opéraitons nécessaires pour l'annotaiton et le nettoyage.

In [9]:
# To run this function proprlely, you need to import modules needed
def CleaningText(text_as_string, language = 'english', reduce = '', list_pos_to_keep = [], Stopwords_to_add = []):
    from nltk.corpus import stopwords

    words = nltk.word_tokenize(text_as_string)
    words_pos = nltk.pos_tag(words, tagset='universal')
    words_pos = [(w, pos) for w, pos in words_pos if w.isalnum()]
    words_pos = [(w.lower(), pos) for w, pos in words_pos]
    
    if reduce == 'stem': 
        from nltk.stem.porter import PorterStemmer
        reduced_words_pos = [(PorterStemmer().stem(w), pos) for w, pos in words_pos]
        
    elif reduce == 'lemma':
        from nltk.stem.wordnet import WordNetLemmatizer
        reduced_words_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
    else:
        import warnings
        reduced_words_pos = words_pos
        warnings.warn("Warning : any reduction was made on words! Please, use \"reduce\" argument to chosse between 'stem' or  'lemma'")
    if list_pos_to_keep:
        reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if pos in list_pos_to_keep]
    else:
        import warnings
        warnings.warn("Warning : any POS filtering was made. Pleae, use \"list_pos_to_keep\" to create a list of POS tag to keep.")
    
    list_stopwords = stopwords.words(language) + Stopwords_to_add
    reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if w not in list_stopwords and len(w) > 1 ]
    return reduced_words_pos



#### <font color = 'E3A440'>*b. Application function nettoyage*</font>

Maintenant, nous pouvons apliquer cette function à chaque fragment de texte.

In [266]:
cleaned_sentences = [CleaningText(sent) for sent in sentences]
print(cleaned_sentences)

[[('eight', 'NUM'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')], [('following', 'ADJ'), ('morning', 'NOUN'), ('nine', 'NUM'), ('arthur', 'NOUN'), ('felt', 'VERB'), ('better', 'ADV')], [('dog', 'NOUN'), ('run', 'NOUN'), ('street', 'NOUN')]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Warning : any reduction was made on words! Please, use "reduce" argument to chosse between 'stem' or  'lemma'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Warning : any POS filtering was made. Pleae, use "list_pos_to_keep" to create a list of POS tag to keep.


In [267]:
cleaned_sentences = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = ['NOUN','ADJ','VERB']) for sent in sentences]
print(cleaned_sentences)

[[('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')], [('following', 'ADJ'), ('morning', 'NOUN'), ('arthur', 'NOUN'), ('felt', 'VERB')], [('dog', 'NOUN'), ('run', 'NOUN'), ('street', 'NOUN')]]


#### <font color = 'E3A440'>*c. Fréquence mots*</font>

Quelle est la fréquence des mots de notre corpus? Pour répondre, nous créons une liste de mots en retirant l'annotation morphosyntaxique.

In [268]:
freqs_in_text = nltk.FreqDist([w for sent in cleaned_sentences for w, pos in sent ])
freqs_in_text

FreqDist({'morning': 2, 'arthur': 2, 'thursday': 1, 'great': 1, 'feel': 1, 'good': 1, 'following': 1, 'felt': 1, 'dog': 1, 'run': 1, ...})

### <font color = 'E3A440'>*3. Vectorisation*</font>

Généralement, pour utiliser le texte dans un contexte d'analyse de données ou d'apprentissage automatique, ce texte doit être transformé dans un objet mathématique approprié. 
Le modèle le plus simple et diffusé est le "bags-of-words", dans lequel chaque fragments de texte (ou chaque mot) est défini par un certain nombre d'unités lexicales qui le caractérise et converti dans une vecteur. Ce modèle appartien à la famille de modèles de la sémantique vectorielle.


The main step in text mining is to convert unstructured textual data into a mathematical model to be used in statistical learning. Thus, we need to create a <font color='E3A440'>**Document-Term matrix**</font>, a matrix $n \times w$, where $n$ is the number of text segments and $w$ is the number of textual features selected.The textual feature can have different nature. In the simplest model, these features correspond to the set of types that resume each token of the corpus. In other terms, $w$ is the number of features that characterize a segment of text. The matrix is generally represented as follows:  
 
$$X = \begin{bmatrix} 
x_{11} & x_{12} & \ldots & x_{1w} \\
\vdots & \vdots       &  \ddots      & \vdots \\ 
x_{n1} & x_{12} & \ldots & x_{nw} \\
\end{bmatrix}
$$ 
 
\\

In [269]:
# Initialisation de l'objet
from nltk.corpus import stopwords

def identity_tokenizer(text):
    return text

# Transforming the word in frequencies
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 1, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 10, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

Utilisation du "vectorizer" avec une liste de listes de mot (et non une liste de tuple de mots-pos).

In [270]:
# Liste de liste de mots:
[[w for w, pos in sent] for sent in cleaned_sentences]

[['thursday', 'morning', 'great', 'arthur', 'feel', 'good'],
 ['following', 'morning', 'arthur', 'felt'],
 ['dog', 'run', 'street']]

In [271]:
# pplication du vectorizer
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_sentences])
print(pd.DataFrame(freq_term_DTM.todense(), columns =  [k for k, v in sorted(vectorized.vocabulary_.items(), key=lambda item: item[1])] ))

   arthur  dog  feel  felt  following  good  great  morning  run  street  \
0       1    0     1     0          0     1      1        1    0       0   
1       1    0     0     1          1     0      0        1    0       0   
2       0    1     0     0          0     0      0        0    1       1   

   thursday  
0         1  
1         0  
2         0  


---
In the next chunk of code, we create another matrix using `scikit-learn` but this time we use the <font color = 'E3A440'>Tf-IDF</font> as weighting scheme of each word. The Tf-IDF is calculated as follows:

$
\begin{align}
&\text{Let}\ t = \text{Term}\\
&\text{Let}\ IDF = \text{Inverse Document Frequency}\\
&    \text{Let}\ TF =\text{Term Frequency}\\[2em]
&    TF \: =\: \frac{\text{term frequency in document}}{\text{total words in document}}\\[1em]
&    IDF(t) \: =\: \log_2\left(\frac{\text{total documents in corpus}}{\text{documents with term}}\right)
\end{align}
$

Then Tf–IDF is calculated as :
$$tfidf( t, d, D ) = tf( t, d ) \times idf( t, D )$$

Thus, we assign the result of the Tf-IDF weighting to the variable named `tfidf_DTM`. 

In [272]:
# Calculate the tfidf matrix
tfidf = TfidfTransformer(norm='l1')
tfidf_DTM = tfidf.fit_transform(freq_term_DTM)
print(pd.DataFrame(tfidf_DTM.todense(), columns =  [k for k, v in sorted(vectorized.vocabulary_.items(), key=lambda item: item[1])] ))

     arthur       dog      feel      felt  following      good     great  \
0  0.137750  0.000000  0.181125  0.000000   0.000000  0.181125  0.181125   
1  0.215994  0.000000  0.000000  0.284006   0.284006  0.000000  0.000000   
2  0.000000  0.333333  0.000000  0.000000   0.000000  0.000000  0.000000   

    morning       run    street  thursday  
0  0.137750  0.000000  0.000000  0.181125  
1  0.215994  0.000000  0.000000  0.000000  
2  0.000000  0.333333  0.333333  0.000000  


# <font color = 'E3A440'> 2. *Exercise : Analyse de sentiment sur Twitter* </font>

L'exercice qui est proposé dans cette section est basé sur une simple chaîne de traitement pour l'**analyse de sentiment** sur de données de Twitter et sur l'**anayse de spécificité lexicales**. 

Le corpus utilisé a été collecté en 2020 par *trackmyhashtag.com* et contient 3 200 tweets pour les 50 profiles les plus suivis de Tweeter. Les données sont en format tabulaire dans un fichier CSV. Pour des raisons pédagogiques, cet exercice prevoit l;'utilisant d'un echantillon aleatoire de 5 000 Tweets.

Dans un premier temps, les données textuelles de 5 000 tweets seront analysées par un module d'analyse de sentiment du module `nltk`. Ensuite, le texte sera pretraité et certaines analyses lexicales serent executées.

Pendans l'exercice, la participant sera invité à remplir les parties manquentes du code qui sont indique avec `...` (trois points).

## <font color = 'E3A440'> 2.1 Présentation de l' exércice </font>

### <font color = 'E3A440'> a. Importer les données </font>

Le fichier avec les données est archivé dans un `.zip` et contient plus de 150 000 tweets. Pour de raisons pédagogiques, nouis importons seulement 5 000 tweets de façon aleatoire. 

In [3]:
ROOT_DIR='Donnees_demystifiees_seance_6/'
DATA_DIR=os.path.join(ROOT_DIR, 'Data')
import zipfile
from datetime import datetime

#Unzips the dataset and gets the TSV dataset
with zipfile.ZipFile(os.path.join(DATA_DIR,'4POINT0_Top_50_tweet_profiles.zip'), 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR)

df = pd.read_pickle(os.path.join(DATA_DIR,'Top_50_tweet_profiles.pkl')).sample(5000, random_state = 5641).reset_index()

Voici les noms de variables disponibles et leur typologie.

In [4]:
df.dtypes

index                                  int64
Tweet Id                              object
Tweet URL                             object
Tweet Posted Time             datetime64[ns]
Tweet Content                         object
Tweet Type                            object
Client                                object
Retweets received                      int64
Likes received                         int64
User Id                               object
Name                                  object
Username                              object
Verified or Non-Verified              object
Profile URL                           object
Protected or Not Protected            object
Profile Account                       object
dtype: object

Voici un exemple:

In [5]:
df.iloc[0]

index                                                                     71560
Tweet Id                                                     656538552327630848
Tweet URL                     https://twitter.com/billboard/status/656538552...
Tweet Posted Time                                           2015-10-20 18:32:43
Tweet Content                 .@JustinBieber, @Skrillex and @Bloodpop's #Sor...
Tweet Type                                                              Retweet
Client                                                       Twitter Web Client
Retweets received                                                         20260
Likes received                                                            22109
User Id                                                                 9695312
Name                                                                  billboard
Username                                                              billboard
Verified or Non-Verified                

### <font color = 'E3A440'> b. Exécuter l'analyse de sentiment</font>

L'objet `SentimentIntensityAnalyzer` est utilisé pour executer l'analyse de sentiment. L'ionbjet doit être initialisé et, ensuite, la fonction `polarity_scores()` peut être appliquée à une chaîne de caractères.

In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Voici trois exemple d'analyse de sentiment. Le résulat de la fonction `polarity_scores()` retourne quatre valeurs: 

 1. `neg` : qui indique le dégrée, dans une echelle de 0 à 1, de sentiment négatif su texte.
 2. `neu` : qui indique le dégrée, dans une echelle de 0 à 1, de sentiment neutre su texte.
 3. `pos` : qui indique le dégrée, dans une echelle de 0 à 1, de sentiment positif su texte.
 4. `compound` : qui contient le valeur d'une métrique composé des trois métriques précedentes et qui va de -1 à 1.



In [7]:
sia.polarity_scores("Wow, NLTK is really powerful!")

{'neg': 0.0, 'neu': 0.295, 'pos': 0.705, 'compound': 0.8012}

In [ ]:
sia.polarity_scores("NLTK is not bad!")

In [ ]:
sia.polarity_scores("NLTK is bad!")

{'neg': 0.655, 'neu': 0.345, 'pos': 0.0, 'compound': -0.5848}

Voici le tweets sur lesquels nous apliquons l'analyse de sentiment:

In [332]:
df['Tweet Content']

0       .@JustinBieber, @Skrillex and @Bloodpop's #Sor...
1       👏 ¡@SergioRamos y @hazardeden10 se encuentran ...
2       Here's how the market may predict the next pre...
3       “Children are magical on road trips. They have...
4       .@MelissaMcCarthy told me about the moment she...
                              ...                        
4995    So saddened to hear of the tragic theatre shoo...
4996    always takes the road less traveled... @ New O...
4997    #HustleHart #MoveWithHart https://t.co/GkQHkhKhR3
4998    This is the letter the US Attorney General sen...
4999    The Week on Instagram | 276\nhttps://t.co/9kIt...
Name: Tweet Content, Length: 5000, dtype: object

Dans le prochain bloc de code, nous exécutons l'anayse de sentiment dur la colonne `Tweet Content`, et nous ajoutons  les résulats obtenus au tableau des données, l'objet nommé `df`.

In [13]:
# Éxecution de l'analyse de sentiment sur tout le corpus
datasent = df.apply(lambda x: sia.polarity_scores(x['Tweet Content']), 1)
df = df.join(pd.DataFrame(list(datasent)))

Le resultas de l'analyse est enregistré sous forme de variables. Voici un exemple:

In [334]:
df.iloc[0]

index                                                                     71560
Tweet Id                                                     656538552327630848
Tweet URL                     https://twitter.com/billboard/status/656538552...
Tweet Posted Time                                           2015-10-20 18:32:43
Tweet Content                 .@JustinBieber, @Skrillex and @Bloodpop's #Sor...
Tweet Type                                                              Retweet
Client                                                       Twitter Web Client
Retweets received                                                         20260
Likes received                                                            22109
User Id                                                                 9695312
Name                                                                  billboard
Username                                                              billboard
Verified or Non-Verified                

Pour rendre simple l'analyse, nous utiliserons seulement la metrique composée `compound` qui est automatiquement calculé par la fonction  `polarity_score()`.

In [340]:
df['compound'].describe()

count    5000.000000
mean        0.199087
std         0.417216
min        -0.972600
25%         0.000000
50%         0.000000
75%         0.557400
max         0.980200
Name: compound, dtype: float64

Pour utiilser la métrique `compound` dans un conteste d'**analyse de spécificité lexicale**, il est nécessaire de constituer des categroies, soit de regourper les tweets sous les categories suivantes: :
 1. `negative` : qui regroupe les tweets contenant un sentiment negatif (`compound` de -1 à -0.5)  
 2. `neu` : qui regroupe les tweets plustôt neutres (`compound` de -0.5 à 0.5)
 3. `positive` : qui regroupe les tweets contenant un sentimnet positif (`compound` plus de 0.5)

In [14]:
# 1 Déterminer les valeurs pour couper la métrique compound
bins = [-1, -0.5, 0.5, 1]
# 2 Déterminer les noms des categoris. NOTEZ que les nombres de noms de catégories doivent être inferieure aux valeur de découpage.
names = ['negative', 'neu', 'positive']
# Exécuter le decoupage avec la fonction 'cut' de pandas.
df['compound_category']  = pd.cut(df['compound'], bins, labels=names, include_lowest =True)

Voici la distribution des tweets par categorie:

In [15]:
Counter(df['compound_category'])

Counter({'neu': 3316, 'positive': 1401, 'negative': 283})

### <font color = 'E3A440'> c. Annotation, nettoyage et vectorisation des tweets </font>

Nous utilisons la fonction écrite précedemement pour nettoyer les unités lexicales de tweets. Pour ce premier test nous conservons seulement les adjectifs.

Cette opération prendra quleque secondes. 

In [335]:
cleaned_tweets = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = ['ADJ'], Stopwords_to_add=['http']) for sent in list(df['Tweet Content'])]

Dans l'étape de vectorisaiton nous retenons les mots qui apparaissent dans au moins 5 documents (`min_df = 5`).

In [342]:
# Initialisation de l'objet
def identity_tokenizer(text):
    return text
# Transforming the word in frequencies
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 5, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 4500, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

In [343]:
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_tweets])
freq_term_DTM

<5000x172 sparse matrix of type '<class 'numpy.int64'>'
	with 2661 stored elements in Compressed Sparse Row format>

### <font color = 'E3A440'> d. Analyse des specificités lexicales </font>

L'analyse de spécificités lexicales permet de metter en evidence les unité lexicales qui sont spécifique à un particulier groupe de données. Dans notre cas, il est possible de identifier les mot qui sont plus fortement associés avec des sentiment positif ou négatif. 

Pour se faire, nous utilisons une métrique très diffusé en lexicometrie, qui est la fonction de vraisemblance (log-likelihood Ratio). La metrique est basée sur cet [article](https://aclanthology.org/J93-1003.pdf). D'autres méthodes peuvent être utilisées, comme l'infiormation mutuelle, le chi2 ou la ponderation Tf-Idf.

In [20]:
def GetLexicalSpecificities(freq_term_DTM, logical_vector):
    # This code ref takes inspiration from this python module : https://pypi.org/project/corpus-toolkit/
    # and its main script:  https://github.com/kristopherkyle/corpus_toolkit/blob/master/corpus_toolkit/corpus_tools.py
    # which is based on this paper: https://aclanthology.org/J93-1003/
    import math
    df_freq_target = pd.DataFrame(np.asarray(freq_term_DTM[logical_vector].sum(0).T).reshape(-1))
    df_freq_target.index = [word for (word,idx) in sorted(vectorized.vocabulary_.items(), key= lambda x:x[1])]
    df_freq_target.columns = ['freq1']
    df_freq_target['freq2'] = np.asarray(freq_term_DTM[~(logical_vector)].sum(0).T).reshape(-1)
    df_freq_target['tot'] = df_freq_target['freq1'] + df_freq_target['freq2']

    df_freq_target['freq1'] = df_freq_target['freq1'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
    df_freq_target['freq2'] = df_freq_target['freq2'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
    #
    df_freq_target['freq1_norm'] = df_freq_target['freq1']/df_freq_target['freq1'].sum() * 1000000
    df_freq_target['freq2_norm'] = df_freq_target['freq2']/df_freq_target['freq2'].sum() * 1000000
    #
    df_freq_target['fraction'] = df_freq_target['freq1_norm'] / df_freq_target['freq2_norm']
    df_freq_target['Log-likelihood Ratio'] = df_freq_target['fraction'].apply(math.log2)
    frequency_threshold = 10 # Insert your frequency threshold as integer
    return df_freq_target[df_freq_target['tot'] > frequency_threshold]['Log-likelihood Ratio'].sort_values(ascending=False).iloc[range(50)]

Poour exécuter l'analyse de spécificité est nécessaire de créaer un vecteur logique (avec de valeurs binaires) qui indique comme `True` la classe pour laquelle nous voulons analyser sa spécifcité lexicale et avec `False`le reste du corpus. 

In [372]:
logical_vector = df['compound_category'] == 'positive'
logical_vector

0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Name: compound_category, Length: 5000, dtype: bool

In [373]:
sum(logical_vector)

1401

Executer la fonciton avec la matrice des fréquences (`freq_term_DTM`) et le vectur logique que nous avons créé plus haut.

In [ ]:
GetLexicalSpecificities(freq_term_DTM, logical_vector)

## <font color = 'E3A440'> 2.2 Exercice </font>

Pendans l'exercice, la participant sera invité à remplir les parties manquentes du code qui sont indique avec `...` (trois points).

Plusieurs manipulations et différent résultats seront demandés. Chaque sous éxercice suit la suivante chaîne de traitement:

1. Annotation et nettoyage des tweets : la participant devra ajuster quelque parametre de la fonction pour choisir un filtrage spécifique
2. Vectorisation:  la participant devra ajuster quelque parametre de la fonction pour choisir un filtrage spécifique
3. Création d'un vecteur logique pour défnir le groupe target et le groupede réference. 
4. Application de la fonction `GetLexicalSpecificities()` pour obtenir les 50 mots les plus spécifiques au groupe target.




### <font color = 'E3A440'> a. Étudfier l'impact du filtrage morphosyntaxique sur les spécificités lexicales </font>

Au point 2.1, seulement les adjectifs ont été étudiés. Faites maintenant une étude sur les noms, adjectifs et verbes et ensuite sur d'autres combination que vous interesse. 
Voici la liste des POS tag existant:

| **POS** | **DESCRIPTION**           | **EXAMPLES**                                      |
| ------- | ------------------------- | ------------------------------------------------- |
| ADJ     | adjective                 | big, old, green, incomprehensible, first      |
| ADP     | adposition                | in, to, during                                |
| ADV     | adverb                    | very, tomorrow, down, where, there            |
| AUX     | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ    | conjunction               | and, or, but                                  |
| CCONJ   | coordinating conjunction  | and, or, but                                  |
| DET     | determiner                | a, an, the                                    |
| INTJ    | interjection              | psst, ouch, bravo, hello                      |
| NOUN    | noun                      | girl, cat, tree, air, beauty                  |
| NUM     | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART    | particle                  | ’s, not                                      |
| PRON    | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN   | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT   | punctuation               | ., (, ), ?                                    |
| SCONJ   | subordinating conjunction | if, while, that                               |
| SYM     | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :)               |
| VERB    | verb                      | run, runs, running, eat, ate, eating          |
| X       | other                     | sfpksdpsxmsa                                  |
| SPACE   | space                     |                                                   |


Inserez la bonne valeur pour l'argument `list_pos_to_keep` afin de pouvoir garder les noms, adjectifs et verbers, ou toute autres combination de POS tag de votre intérêt

In [10]:
# 1. Annotation and cleaning : ADD adjective and verbs as POS tag to keep
cleaned_tweets = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = [...], Stopwords_to_add=['http']) for sent in list(df['Tweet Content'])]

Changez les parametres `min_df` afin de ne pas depasser **750 mots** de votre matrice <font color='E3A440'>**Document-Term matrix**</font>, qui est enregistrée dans l'objet `freq_term_DTM`.

Notez bien que dans cette fonction le paramtre `ngram_range` est configuré pour avoir les unigrams et les bigrams (sa valeur est : `(1,2)`).

In [40]:
# 2. Vectorisation
def identity_tokenizer(text):
    return text
## 2.1 initialise with parameters : 
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 10, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 4500, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 2), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

#
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_tweets])

freq_term_DTM

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


<5000x727 sparse matrix of type '<class 'numpy.int64'>'
	with 20057 stored elements in Compressed Sparse Row format>

En utilisant le travail fait déjà au point **b.** de la section **2.1**, choisissez la categorie de sentiment pour laquelle vous voulez étudier la spécioficité lexicale ex.  `negative` or `positive`.

In [41]:
logical_vector = df['compound_category'] == ...

En utilisant la fonction définie au point **d.** de la section **2.1**, ajoutez les arguments fondamentales de la focntion, soit la matrice  <font color='E3A440'>**Document-Term matrix**</font> et le **vecteur logique** créé dans le bloc de code précedent.

In [42]:
GetLexicalSpecificities(freq_term_DTM, logical_vector)

beautiful         28.750313
congratulation    28.498775
super             28.106457
honored           27.691420
haha              27.565889
amazing            6.002708
free               5.710942
congrats           5.376523
happy              5.299217
award              4.852961
happy birthday     4.553400
awesome            4.553400
great              4.528526
thank love         4.437923
fun                4.397281
perfect            4.312392
strong             4.312392
love               4.123814
gift               3.759851
best               3.743731
loved              3.589926
birthday           3.569168
win                3.553400
progress           3.553400
honor              3.437923
wow                3.355461
wish               3.338387
special            3.312392
hilarious          3.174889
peace              3.174889
thank much         3.100888
kind               3.075353
hope               3.062414
proud              3.045606
friend             2.625550
energy             2

### <font color = 'E3A440'> b. Étudier de nouvelles categories basée sur le nombre de Retweet </font>

Sur tweeter il est possible de Retweetter un Tweet existant. Le nombre de retweet peut être consideré un indicateur su suivi qu'un tweet a obtenu. En accomplissant les mêmes étapes appris au long de cet atelier, quels sont les spécificité lexicales de tweet qui ont eu un très grand suivi? 

Voici la distribution de la colonne `Retweets received`.

In [43]:
df['Retweets received'].describe()

count      5000.00000
mean       6181.56700
std       21201.34775
min           0.00000
25%         161.00000
50%         741.00000
75%        3241.00000
max      449711.00000
Name: Retweets received, dtype: float64

En suivant les percentile qui sont affiché dans la distribution de la colonne `Retweets received` (résultat du bloc de code précedent), ajouté le valeur de decoupage manquante dans la liste des "bins". 
Nous diviserons le nombre de Retweet en quatres categorie: 
1. `low`, qui regroupe les tweets ayant récu un faible suivi
2. `medium`, qui regroupe les tweets ayant récu un suivi moyen
3. `high`, qui regroupe les tweets ayant récu un grand suivi
4. `very_high`, qui regroupe les tweets ayant récu un très grand suivi

In [44]:
bins = [-np.inf, 161, ..., ..., 449711]
names = ['low', 'medium', 'high', 'very_high']
df['Retweets_received_category']  = pd.cut(df['Retweets received'], bins, labels=names, include_lowest =True)

Choisir la categorie target pour laquelle analyser les spécificités lexicale. La vlauer doit être une des quatres valeurs contenus dans la colonne `Retweets_received_category`générée dans le bloc de code précedent.

In [45]:
logical_vector = df['Retweets_received_category'] == ...

Éxecutez l'analyse de spécificités.

In [49]:
GetLexicalSpecificities(freq_term_DTM, logical_vector)

realdonaldtrump    7.083156
republican         6.083156
democrat           6.083156
military           4.538835
witness            4.498193
impeachment        4.407591
nothing            4.320655
worst              3.953873
fact               3.623724
taylor             3.591303
done               3.439300
speaker            3.346190
president          3.135043
kevin              3.024262
call               3.024262
american           2.953873
congress           2.913231
government         2.886759
court              2.886759
iran               2.835228
job                2.761228
local              2.761228
money              2.761228
case               2.673765
trying             2.673765
trump              2.673765
lost               2.645750
public             2.645750
hearing            2.591303
release            2.591303
possible           2.591303
continues          2.591303
wrong              2.591303
able               2.591303
speech             2.591303
event              2

### <font color = 'E3A440'> c. Étudier les language spécifique des profiles Tweeter </font>

Dans le prochain éxercice, sélectionnez deux ou trois profiles Tweeter de votre choix et comparez les specificité lexicales en étudiant plusieurs combination dePOS tag. Quels sont les grandes différenes lexiclaers entres les profies que vous avez choisie? 

Voici la liste complète des profiles présent dans le corpus et enregistrés sous la colonne `Profile Account` et les nombre dce tweets par profile.

In [62]:
Counter(df['Profile Account'])

Counter({'justinbieber': 109,
         'realmadrid': 106,
         'cnn': 94,
         'nytimes': 109,
         'theellenshow': 95,
         'jlo': 89,
         'sportscenter': 106,
         'louis_tomlinson': 104,
         'shakira': 107,
         'rihanna': 98,
         'bbcbreaking': 106,
         'jtimberlake': 115,
         'realdonaldtrump': 105,
         'ladygaga': 87,
         'selenagomez': 112,
         'britneyspears': 118,
         'liampayne': 93,
         'neymarjr': 115,
         'cnnbrk': 97,
         'kingjames': 103,
         'twitter': 97,
         'cristiano': 111,
         'niallofficial': 109,
         'narendramodi': 100,
         'drake': 56,
         'espn': 99,
         'liltunechi': 70,
         'oprah': 100,
         'barackobama': 95,
         'iamsrk': 102,
         'arianagrande': 105,
         'imvkohli': 62,
         'jimmyfallon': 116,
         'harry_styles': 98,
         'instagram': 97,
         'beingsalmankhan': 100,
         'youtube': 100,
    

In [64]:
# 1. Annotation and cleaning : ADD adjective and verbs as POS tag to keep
cleaned_tweets = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = [...], Stopwords_to_add=['http']) for sent in list(df['Tweet Content'])]

# 2. Vectorisation
def identity_tokenizer(text):
    return text
## 2.1 initialise with parameters : 
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 10, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 4500, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

#
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_tweets])

freq_term_DTM

<5000x437 sparse matrix of type '<class 'numpy.int64'>'
	with 10635 stored elements in Compressed Sparse Row format>

In [69]:
logical_vector = df['Profile Account'] == 'nasa'
GetLexicalSpecificities(freq_term_DTM, logical_vector)

artemis       31.795959
spacecraft    31.248471
nasa           6.994975
mar            6.753967
moon           6.248731
earth          6.220535
program        6.169004
space          6.031501
mission        5.432039
launch         5.294535
test           5.072143
science        4.972607
system         4.809108
et             4.500986
detail         4.124610
planet         3.809108
data           3.487180
flight         3.294535
tune           3.046608
camera         2.879498
meet           2.835103
question       2.631570
future         2.594095
medium         2.207072
summer         2.072143
share          2.046608
help           1.972607
honor          1.972607
name           1.972607
decade         1.879498
journey        1.835103
hour           1.835103
step           1.835103
company        1.835103
plan           1.835103
star           1.709573
minute         1.631570
photo          1.616463
nation         1.594095
florida        1.594095
june           1.487180
challenge      1

## <font color = 'E3A440'> 2.3 NOTES PERSONELLES: </font>

-----

-----